# Chapter 9: Lists

## Data.Char

In [1]:
import Data.Char

In [2]:
:type isUpper

isUpper :: Char -> Bool

In [3]:
:type toUpper

toUpper :: Char -> Char

In [4]:
filter isUpper "HbEfLrLxO"

"HELLO"

In [5]:
capitalize :: String -> String
capitalize [] = []
capitalize (x:xs) = toUpper x : xs

capitalize "julie"

"Julie"

In [6]:
upperAll :: String -> String
upperAll [] = []
upperAll (x:xs) = toUpper x : upperAll xs

upperAll "woot"

Line 2: Use map
Found:
upperAll [] = []
upperAll (x : xs) = toUpper x : upperAll xs
Why not:
upperAll xs = map toUpper xs

"WOOT"

In [7]:
:type head

head :: forall a. [a] -> a

In [8]:
getCapitalHead :: String -> Char
getCapitalHead = toUpper . head

getCapitalHead "julie"

'J'

## Ciphers

In [9]:
:t chr

chr :: Int -> Char

In [10]:
:t ord

ord :: Char -> Int

In [11]:
base = ord 'a'
end = ord 'z'
size = end - base + 1

In [12]:
import Data.Maybe

newtype Letter = Letter Char deriving Show

unLetter :: Letter -> Char
unLetter (Letter x) = x

getLetter :: Char -> Maybe Letter
getLetter x
    | l < base = Nothing
    | l > end = Nothing
    | otherwise = (Just . Letter) x
    where l = ord x

getLetterFromInt :: Int -> Maybe Letter
getLetterFromInt = getLetter . chr

unsafeGetLetter :: Char -> Letter
unsafeGetLetter = fromJust . getLetter

unsafeGetLetterFromInt :: Int -> Letter
unsafeGetLetterFromInt = fromJust . getLetterFromInt

caesarLetter :: Int -> Letter -> Letter
caesarLetter shift (Letter x) = (unsafeGetLetterFromInt . f . ord) x where
    f i = (i - base + shift) `mod` 100 + base
    
caesarLetter 3 (Letter 'a')

Letter 'd'

In [13]:
getLetters :: String -> Maybe [Letter]
getLetters = traverse getLetter

unsafeGetLetters :: String -> [Letter]
unsafeGetLetters = fromJust . getLetters

unLetters :: [Letter] -> String
unLetters = fmap unLetter

caesar :: Int -> [Letter] -> [Letter]
caesar shift = fmap (caesarLetter shift)

unLetters $ caesar 3 (unsafeGetLetters "qwer")

"tzhu"

In [14]:
uncaesar :: Int -> [Letter] -> [Letter]
uncaesar shift = caesar (-shift)

unLetters $ uncaesar 3 $ caesar 3 (unsafeGetLetters "qwer")

"qwer"

## Writing your own standard functions

In [15]:
and []
or []

True

False

In [16]:
myOr :: [Bool] -> Bool
myOr [] = False
myOr (x:xs) = x || myOr xs

myOr [False, True]

Line 2: Use foldr
Found:
myOr [] = False
myOr (x : xs) = x || myOr xs
Why not:
myOr xs = foldr (||) False xs

True

In [17]:
myOr :: [Bool] -> Bool
myOr = foldr (||) False

Line 2: Use or
Found:
foldr (||) False
Why not:
or

In [18]:
:t any

any :: forall (t :: * -> *) a. Foldable t => (a -> Bool) -> t a -> Bool

In [19]:
any (>0) []

False

In [20]:
myAny :: (a -> Bool) -> [a] -> Bool
myAny f [] = False
myAny f (x:xs) = f x || myAny f xs

myAny (>0) []

Line 2: Use foldr
Found:
myAny f [] = False
myAny f (x : xs) = f x || myAny f xs
Why not:
myAny f xs = foldr ((||) . f) False xs

False

In [21]:
myAny :: (a -> Bool) -> [a] -> Bool
myAny f = foldr ((||) . f) False

myAny (>0) [-1, -2]
myAny (>0) [-1, 3]

False

True

In [22]:
myAny :: (a -> Bool) -> [a] -> Bool
myAny f = or . fmap f

myAny (>0) [-1, -2]
myAny (>0) [-1, 3]

False

True

In [23]:
myElem :: Eq a => a -> [a] -> Bool
myElem _ [] = False
myElem x (y:ys)
    | x == y = True
    | otherwise = myElem x ys
    
myElem 12 [12, 134]
myElem 12 [11, 134]

True

False

In [24]:
myElem :: Eq a => a -> [a] -> Bool
myElem x = any (==x)

myElem 12 [12, 134]
myElem 12 [11, 134]

Line 2: Use elem
Found:
any (== x)
Why not:
elem x

True

False

In [25]:
myReverse :: [a] -> [a]
myReverse = f [] where
    f acc [] = acc
    f acc (x:xs) = f (x:acc) xs
    
myReverse "qwer"

Line 3: Use foldl
Found:
f acc [] = acc
f acc (x : xs) = f (x : acc) xs
Why not:
f acc xs = foldl (flip (:)) acc xs

"rewq"

In [26]:
myReverse :: [a] -> [a]
myReverse = foldl (flip (:)) []
    
myReverse "qwer"
myReverse []

"rewq"

[]

In [27]:
squish :: [[a]] -> [a]
squish [] = []
squish (xs:xss) = xs ++ squish xss

squish [[1, 2], [3, 4]]

Line 2: Use foldr
Found:
squish [] = []
squish (xs : xss) = xs ++ squish xss
Why not:
squish xss = foldr (++) [] xss

[1,2,3,4]

In [28]:
squish :: [[a]] -> [a]
squish = foldr (++) []

squish [[1, 2], [3, 4]]

Line 2: Use concat
Found:
foldr (++) []
Why not:
concat

[1,2,3,4]

In [29]:
duplicate :: Char -> String
duplicate c = [c, c]

squishMap :: (a -> [b]) -> [a] -> [b]
squishMap _ [] = []
squishMap f (x:xs) = f x ++ squishMap f xs

squishMap duplicate "qwer"

"qqwweerr"

In [30]:
squishMap :: (a -> [b]) -> [a] -> [b]
squishMap f = foldr ((++) . f) []

squishMap duplicate "qwer"

Line 2: Use concatMap
Found:
foldr ((++) . f) []
Why not:
concatMap f

"qqwweerr"

In [31]:
squishAgain :: [[a]] -> [a]
squishAgain = squishMap id

squishAgain [[1, 2], [3, 4]]

[1,2,3,4]

In [32]:
:info Ordering

data Ordering = LT | EQ | GT 	-- Defined in ‘GHC.Types’
instance Eq Ordering -- Defined in ‘GHC.Classes’
instance Ord Ordering -- Defined in ‘GHC.Classes’
instance Show Ordering -- Defined in ‘GHC.Show’
instance Bounded Ordering -- Defined in ‘GHC.Enum’
instance Monoid Ordering -- Defined in ‘GHC.Base’
instance Semigroup Ordering -- Defined in ‘GHC.Base’
instance Read Ordering -- Defined in ‘GHC.Read’
instance Enum Ordering -- Defined in ‘GHC.Enum’

In [33]:
import Data.List (maximumBy)

greater :: Int -> Int -> Ordering
greater x y
    | x > y = GT
    | x < y = LT
    | x == y = EQ

maximumBy greater [1, 2, 3, 4]
maximumBy undefined []

4

: 

In [34]:
qwer :: Maybe Int -> Int
qwer x | Just v <- x = v

qwer (Just 10)

10

In [35]:
myMaximumBy :: (a -> a -> Ordering) -> [a] -> a
myMaximumBy _ [] = undefined
myMaximumBy _ [x] = x
myMaximumBy f (x:xs) = case f x x' of
    LT -> x'
    _ -> x
    where x' = myMaximumBy f xs

myMaximumBy greater [1, 2, 3, 4]
myMaximumBy undefined []

4

: 

In [36]:
:type foldl1

foldl1 :: forall (t :: * -> *) a. Foldable t => (a -> a -> a) -> t a -> a

In [37]:
myMaximumBy :: (a -> a -> Ordering) -> [a] -> a
myMaximumBy f = foldl1 select where
    select x y = case f x y of
        LT -> y
        _ -> x
        
myMaximumBy greater [1, 2, 3, 4]
myMaximumBy undefined []        

4

: 

In [38]:
myMinimumBy :: (a -> a -> Ordering) -> [a] -> a
myMinimumBy f = foldl1 select where
    select x y = case f x y of
        GT -> y
        _ -> x
        
myMinimumBy greater [1, 2, 3, 4]
myMinimumBy undefined []        

1

: 

In [39]:
:t compare

compare :: forall a. Ord a => a -> a -> Ordering

In [40]:
myMaximum :: Ord a => [a] -> a
myMaximum = myMaximumBy compare

myMaximum [1, 2, 4, 3]

4

In [41]:
myMinimum :: Ord a => [a] -> a
myMinimum = myMinimumBy compare

myMinimum [2, 1, 4, 3]

1